# Example: Properties

#### Imports

In [1]:
import numpy as np
import torch
import time
from pathlib import Path

import dxtb

#### Functions

#### dxtb calculation

In [2]:
f = Path(globals()['_dh'][0]) / "molecules" / "taxol.coord"
atoms, xyz = dxtb.io.read_coord(f)

In [3]:
device = torch.device("cpu")
dd: dxtb._types.DD = {"device": device, "dtype": torch.double}

numbers = torch.tensor(atoms, device=device)
positions = torch.tensor(xyz, **dd)
charge = torch.tensor(0.0, **dd)

mol = dxtb.mol.external.M(numbers, positions)

par = dxtb.GFN1_XTB
ihelp = dxtb.IndexHelper.from_numbers(numbers, dxtb.param.get_elem_angular(par.element))

In [4]:
from dxtb.integral.driver.libcint import OverlapLibcint, IntDriverLibcint
from dxtb.integral.driver.pytorch import OverlapPytorch, IntDriverPytorch, IntDriverPytorchNoAnalytical
from dxtb.integral.driver.pytorch.impls import overlap as overlap_fwd

drv_py = IntDriverPytorch(numbers, par, ihelp, **dd)
drv_py.setup(positions)

drv_py2 = IntDriverPytorchNoAnalytical(numbers, par, ihelp, **dd)
drv_py2.setup(positions)

drv_lc = IntDriverLibcint(numbers, par, ihelp, **dd)
drv_lc.setup(positions)

ovlp_py = OverlapPytorch(**dd)
ovlp_lc = OverlapLibcint(**dd)

# create basis
basis = dxtb.Basis(numbers, par, ihelp, **dd)
ubas = dxtb.Basis(torch.unique(numbers), par, ihelp, **dd)

In [5]:
from dxtb.utils.timing import timings

n=1

@timings(n)
def run_sauto():
    """Explicit call of forward function."""
    return overlap_fwd(positions, ubas, ihelp)

@timings(n)
def run_sdxtb():
    return ovlp_py.build(drv_py)

@timings(n)
def run_slibc():
    return ovlp_lc.build(drv_lc)

@timings(n)
def run_pyscf():
    s = torch.tensor(mol.intor("int1e_ovlp"), **dd)
    norm = torch.pow(s.diagonal(dim1=-1, dim2=-2), -0.5)
    return torch.einsum("...ij,...i,...j->...ij", s, norm, norm)


s1 = run_sauto()
s2 = run_sdxtb()
s3 = run_slibc()
s4 = run_pyscf()

assert s1.shape == s2.shape
assert s1.shape == s3.shape
assert s1.shape == s4.shape

0 (tensor(0), tensor(0)) True
1 (tensor(0), tensor(0)) True
2 (tensor(0), tensor(0)) True
3 (tensor(0), tensor(0)) True
4 (tensor(0), tensor(1)) True
5 (tensor(0), tensor(0)) True
6 (tensor(0), tensor(1)) True
7 (tensor(0), tensor(0)) True
8 (tensor(0), tensor(0)) True
9 (tensor(0), tensor(1)) True
10 (tensor(0), tensor(0)) True
11 (tensor(0), tensor(0)) True
12 (tensor(0), tensor(1)) True
13 (tensor(0), tensor(1)) True
14 (tensor(0), tensor(1)) True
15 (tensor(0), tensor(0)) True
16 (tensor(0), tensor(0)) True
17 (tensor(0), tensor(1)) True
18 (tensor(0), tensor(1)) True
19 (tensor(0), tensor(1)) True
20 (tensor(0), tensor(0)) True
21 (tensor(0), tensor(1)) True
22 (tensor(0), tensor(1)) True
23 (tensor(0), tensor(0)) True
24 (tensor(0), tensor(1)) True
25 (tensor(0), tensor(1)) True
26 (tensor(0), tensor(0)) True
27 (tensor(0), tensor(1)) True
28 (tensor(1), tensor(0)) False
29 (tensor(1), tensor(0)) False
30 (tensor(1), tensor(0)) False
31 (tensor(1), tensor(0)) False
32 (tensor(1),

In [6]:
s1 = run_sauto()


0 (tensor(0), tensor(0)) True
1 (tensor(0), tensor(0)) True


2 (tensor(0), tensor(0)) True
3 (tensor(0), tensor(0)) True
4 (tensor(0), tensor(1)) True
5 (tensor(0), tensor(0)) True
6 (tensor(0), tensor(1)) True
7 (tensor(0), tensor(0)) True
8 (tensor(0), tensor(0)) True
9 (tensor(0), tensor(1)) True
10 (tensor(0), tensor(0)) True
11 (tensor(0), tensor(0)) True
12 (tensor(0), tensor(1)) True
13 (tensor(0), tensor(1)) True
14 (tensor(0), tensor(1)) True
15 (tensor(0), tensor(0)) True
16 (tensor(0), tensor(0)) True
17 (tensor(0), tensor(1)) True
18 (tensor(0), tensor(1)) True
19 (tensor(0), tensor(1)) True
20 (tensor(0), tensor(0)) True
21 (tensor(0), tensor(1)) True
22 (tensor(0), tensor(1)) True
23 (tensor(0), tensor(0)) True
24 (tensor(0), tensor(1)) True
25 (tensor(0), tensor(1)) True
26 (tensor(0), tensor(0)) True
27 (tensor(0), tensor(1)) True
28 (tensor(1), tensor(0)) False
29 (tensor(1), tensor(0)) False
30 (tensor(1), tensor(0)) False
31 (tensor(1), tensor(0)) False
32 (tensor(1), tensor(0)) False
33 (tensor(1), tensor(0)) False
34 (tensor

In [7]:
s2 = run_sdxtb()

0 (tensor(0), tensor(0)) True


1 (tensor(0), tensor(0)) True
2 (tensor(0), tensor(0)) True
3 (tensor(0), tensor(0)) True
4 (tensor(0), tensor(1)) True
5 (tensor(0), tensor(0)) True
6 (tensor(0), tensor(1)) True
7 (tensor(0), tensor(0)) True
8 (tensor(0), tensor(0)) True
9 (tensor(0), tensor(1)) True
10 (tensor(0), tensor(0)) True
11 (tensor(0), tensor(0)) True
12 (tensor(0), tensor(1)) True
13 (tensor(0), tensor(1)) True
14 (tensor(0), tensor(1)) True
15 (tensor(0), tensor(0)) True
16 (tensor(0), tensor(0)) True
17 (tensor(0), tensor(1)) True
18 (tensor(0), tensor(1)) True
19 (tensor(0), tensor(1)) True
20 (tensor(0), tensor(0)) True
21 (tensor(0), tensor(1)) True
22 (tensor(0), tensor(1)) True
23 (tensor(0), tensor(0)) True
24 (tensor(0), tensor(1)) True
25 (tensor(0), tensor(1)) True
26 (tensor(0), tensor(0)) True
27 (tensor(0), tensor(1)) True
28 (tensor(1), tensor(0)) False
29 (tensor(1), tensor(0)) False
30 (tensor(1), tensor(0)) False
31 (tensor(1), tensor(0)) False
32 (tensor(1), tensor(0)) False
33 (tensor(1

In [8]:
s1 = run_sauto()

0 (tensor(0), tensor(0)) True
1 (tensor(0), tensor(0)) True
2 (tensor(0), tensor(0)) True
3 (tensor(0), tensor(0)) True
4 (tensor(0), tensor(1)) True
5 (tensor(0), tensor(0)) True
6 (tensor(0), tensor(1)) True
7 (tensor(0), tensor(0)) True
8 (tensor(0), tensor(0)) True
9 (tensor(0), tensor(1)) True
10 (tensor(0), tensor(0)) True
11 (tensor(0), tensor(0)) True
12 (tensor(0), tensor(1)) True
13 (tensor(0), tensor(1)) True
14 (tensor(0), tensor(1)) True
15 (tensor(0), tensor(0)) True
16 (tensor(0), tensor(0)) True
17 (tensor(0), tensor(1)) True
18 (tensor(0), tensor(1)) True
19 (tensor(0), tensor(1)) True
20 (tensor(0), tensor(0)) True
21 (tensor(0), tensor(1)) True
22 (tensor(0), tensor(1)) True
23 (tensor(0), tensor(0)) True
24 (tensor(0), tensor(1)) True
25 (tensor(0), tensor(1)) True
26 (tensor(0), tensor(0)) True
27 (tensor(0), tensor(1)) True
28 (tensor(1), tensor(0)) False
29 (tensor(1), tensor(0)) False
30 (tensor(1), tensor(0)) False
31 (tensor(1), tensor(0)) False
32 (tensor(1),

### get_gradient

In [9]:
@timings()
def run_sdxtb_grad():
    return ovlp_py.get_gradient(drv_py)

@timings()
def run_slibc_grad():
    return ovlp_lc.get_gradient(drv_lc)

@timings()
def run_pyscf_grad():
    # not normalized
    ovlp = torch.tensor(mol.intor("int1e_ipovlp"), **dd)
    return torch.einsum("xij->ijx", ovlp)


s1 = run_sdxtb_grad()
s2 = run_slibc_grad()
s3 = run_pyscf_grad()

assert s1.shape == s2.shape
assert s1.shape == s3.shape


1.4748 
0.1254 
0.1259 


### Autograd utils

In [10]:
repeats = 1

def nth_derivative(f, wrt, n=1):
    create_graph = False if n == 1 else True
    for _ in range(n):
        grads, = torch.autograd.grad(f, wrt, create_graph=create_graph)
        f = grads.sum()

    return grads # type: ignore

def print_average(times):
    print()
    if repeats > 1:
        avg_time = sum(times) / len(times)
        print(f"Average ({repeats}): {avg_time:2.4f} sec")

def time_operation(operation, n):
    times = []
    for _ in range(repeats):
        s = operation().sum()  # Execute the operation passed as an argument
        ts = time.perf_counter()
        ds = nth_derivative(s, positions, n=n)
        te = time.perf_counter()
        times.append(te - ts)
        print(f"{te - ts:2.4f}", end=" ")
    
    print_average(times)

### 1st Derivative

In [11]:
positions = positions.detach().clone().requires_grad_(True)
drv_py.setup(positions)
drv_py2.setup(positions)
drv_lc.setup(positions)

n = 1
time_operation(lambda: overlap_fwd(positions, ubas, ihelp), n)
time_operation(lambda: ovlp_py.build(drv_py), n)
time_operation(lambda: ovlp_py.build(drv_py2), n)
time_operation(lambda: ovlp_lc.build(drv_lc), n)


0 (tensor(0), tensor(0)) True
1 (tensor(0), tensor(0)) True


2 (tensor(0), tensor(0)) True
3 (tensor(0), tensor(0)) True
4 (tensor(0), tensor(1)) True
5 (tensor(0), tensor(0)) True
6 (tensor(0), tensor(1)) True
7 (tensor(0), tensor(0)) True
8 (tensor(0), tensor(0)) True
9 (tensor(0), tensor(1)) True
10 (tensor(0), tensor(0)) True
11 (tensor(0), tensor(0)) True
12 (tensor(0), tensor(1)) True
13 (tensor(0), tensor(1)) True
14 (tensor(0), tensor(1)) True
15 (tensor(0), tensor(0)) True
16 (tensor(0), tensor(0)) True
17 (tensor(0), tensor(1)) True
18 (tensor(0), tensor(1)) True
19 (tensor(0), tensor(1)) True
20 (tensor(0), tensor(0)) True
21 (tensor(0), tensor(1)) True
22 (tensor(0), tensor(1)) True
23 (tensor(0), tensor(0)) True
24 (tensor(0), tensor(1)) True
25 (tensor(0), tensor(1)) True
26 (tensor(0), tensor(0)) True
27 (tensor(0), tensor(1)) True
28 (tensor(1), tensor(0)) False
29 (tensor(1), tensor(0)) False
30 (tensor(1), tensor(0)) False
31 (tensor(1), tensor(0)) False
32 (tensor(1), tensor(0)) False
33 (tensor(1), tensor(0)) False
34 (tensor

### 2nd Derivative

In [12]:
positions = positions.detach().clone().requires_grad_(True)
drv_py.setup(positions)
drv_py2.setup(positions)
drv_lc.setup(positions)

n = 2
time_operation(lambda: overlap_fwd(positions, ubas, ihelp), n)
time_operation(lambda: ovlp_py.build(drv_py), n)
time_operation(lambda: ovlp_py.build(drv_py2), n)
time_operation(lambda: ovlp_lc.build(drv_lc), n)

0 (tensor(0), tensor(0)) True
1 (tensor(0), tensor(0)) True
2 (tensor(0), tensor(0)) True
3 (tensor(0), tensor(0)) True
4 (tensor(0), tensor(1)) True
5 (tensor(0), tensor(0)) True
6 (tensor(0), tensor(1)) True
7 (tensor(0), tensor(0)) True
8 (tensor(0), tensor(0)) True
9 (tensor(0), tensor(1)) True
10 (tensor(0), tensor(0)) True
11 (tensor(0), tensor(0)) True
12 (tensor(0), tensor(1)) True
13 (tensor(0), tensor(1)) True
14 (tensor(0), tensor(1)) True
15 (tensor(0), tensor(0)) True
16 (tensor(0), tensor(0)) True
17 (tensor(0), tensor(1)) True
18 (tensor(0), tensor(1)) True
19 (tensor(0), tensor(1)) True
20 (tensor(0), tensor(0)) True
21 (tensor(0), tensor(1)) True
22 (tensor(0), tensor(1)) True
23 (tensor(0), tensor(0)) True
24 (tensor(0), tensor(1)) True
25 (tensor(0), tensor(1)) True
26 (tensor(0), tensor(0)) True
27 (tensor(0), tensor(1)) True
28 (tensor(1), tensor(0)) False
29 (tensor(1), tensor(0)) False
30 (tensor(1), tensor(0)) False
31 (tensor(1), tensor(0)) False
32 (tensor(1),

KeyboardInterrupt: 

### 3rd Derivative

In [ ]:
drv_py.setup(positions)
drv_py2.setup(positions)
drv_lc.setup(positions)

n = 3
time_operation(lambda: overlap_fwd(positions, ubas, ihelp), n)
time_operation(lambda: ovlp_py.build(drv_py), n)
time_operation(lambda: ovlp_py.build(drv_py2), n)
time_operation(lambda: ovlp_lc.build(drv_lc), n)

times 0.014330195030197501 1.3871359848417342
1.4019 


KeyboardInterrupt: 

In [ ]:
repeats = 5

def print_averages(times):
    num_segments = len(times[0])
    print("--------------------")
    for i in range(num_segments):
        avg_time = sum(time[i] for time in times) / len(times)
        print(f"{avg_time:2.4f}", end=" ")
    print()

def measure_times(operation, positions, repeats):
    times = []
    print("\nenergy 1st    2nd    3rd")
    for _ in range(repeats):
        pos = positions.clone().detach().requires_grad_(True)
        t0 = time.perf_counter()

        s = operation(pos).sum()
        t1 = time.perf_counter()
        print(f"{t1 - t0:2.4f}", end=" ")

        (ds,) = torch.autograd.grad(s, pos, create_graph=True)
        t2 = time.perf_counter()
        print(f"{t2 - t1:2.4f}", end=" ")
        
        dds, = torch.autograd.grad(ds.sum(), pos, create_graph=True)
        t3 = time.perf_counter()
        print(f"{t3 - t2:2.4f}", end=" ")

        ddds, = torch.autograd.grad(dds.sum(), pos)
        t4 = time.perf_counter()
        print(f"{t4 - t3:2.4f}", end=" ")
        
        dddds, = torch.autograd.grad(ddds.sum(), pos)
        t5 = time.perf_counter()
        print(f"{t5 - t4:2.4f}", end=" ")
        times.append([t1 - t0, t2 - t1, t3 - t2, t4 - t3, t5 - t4])

        print()

    return times

def ovlp_fwd_func(pos):
    return overlap_fwd(pos, ubas, ihelp)

def ovlp_py_func(pos):
    drv_py.setup(pos)
    return ovlp_py.build(drv_py)

def ovlp_py2_func(pos):
    drv_py2.setup(pos)
    return ovlp_py.build(drv_py2)

def ovlp_lc_func(pos):
    drv_lc.setup(pos)
    return ovlp_lc.build(drv_lc)

# Call the function with different operations
times = measure_times(lambda pos: ovlp_fwd_func(pos), positions, repeats)
print_averages(times)

times = measure_times(lambda pos: ovlp_py_func(pos), positions, repeats)
print_averages(times)

# Replace with other operations as needed
times = measure_times(lambda pos: ovlp_py2_func(pos), positions, repeats)
print_averages(times)

times = measure_times(lambda pos: ovlp_lc_func(pos), positions, repeats)
print_averages(times)


energy 1st    2nd    3rd
tensor([[ 0.0000,  0.0000,  0.0000],
        [-1.1908, -1.1908, -1.1908],
        [ 1.1908,  1.1908, -1.1908],
        [ 1.1908, -1.1908,  1.1908],
        [-1.1908,  1.1908,  1.1908]], dtype=torch.float64, requires_grad=True)
70.0
tensor([[False, False, False, False,  True,  True,  True,  True,  True,  True,
          True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True,
          True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True,
          True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True,
          True,  True],
        [ True,  True,  True,  True, False, False,  True,  True,  True,  True,
          True,  True],
        [ True,  True,  True,  True, False, False,  True,  True,  True,  True,
          True,  True],
        [ True,  True,  True,  True,  True,  True, False, False,  True,  True,
          True,  True],
        [ True,  True

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn